<a href="https://colab.research.google.com/github/VRamBalla/BME590-Protein-Design/blob/main/Proj1_Final_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mounting

In [ ]:
# Mount to drive
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


#Importing Libraries

In [ ]:
!pip install exrex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import sklearn
import pickle
import exrex
import re

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import InputLayer, Conv2D, Dropout, MaxPooling2D, Flatten, Dense
from tensorflow.keras import layers
from keras import models
from tqdm import tqdm
import math

#Data Processing

First, the dataset is loaded in using pandas. This dataset contains protein sequence information and the 8 nucleotide PAM sequence. A dataframe called data_8nt is created, and only the protein and PAM sequences from the dataset are carried over. Additionally, the PAM sequences is split into 8 additional columns, where each column represents one nucleotide in the PAM sequence. Thus, the final dataframe we are working with contains 11 columns: 1 for the amino acid sequence, 1 for the PAM sequence, 8 for each nucleotide position, and 1 for the amino acid sequence length.

Below is the code used to generate the final cleaned data set that was used in model training. The code is commented out because it only needs to be run once since the cleaned data was written as a csv, so it can now just be loaded in.

In [ ]:
# data8 = pd.read_csv('/content/drive/My Drive/team_5/project_1/Data/PAM_data8_ranked.csv')
# casper_data = pd.read_csv('/content/drive/My Drive/team_5/project_1/Data/additional_PAM.csv')
# casper_data = casper_data.rename(columns={"Amino Acid Sequence": "AA Sequence"})
# data_8nt = pd.DataFrame()
# data_8nt['AA Sequence'] = data8['Sequence']
# data_8nt['PAM'] = data8['consensus PAM']
# data_8nt = data_8nt.append(casper_data)

# l = []
# for i in data_8nt['AA Sequence']:
#   l.append(len(i))
  
# data_8nt['Sequence Length'] = l

# data_8nt['nt1'] = data_8nt.PAM.str.split('',expand=True)[1]
# data_8nt['nt2'] = data_8nt.PAM.str.split('',expand=True)[2]
# data_8nt['nt3'] = data_8nt.PAM.str.split('',expand=True)[3]
# data_8nt['nt4'] = data_8nt.PAM.str.split('',expand=True)[4]
# data_8nt['nt5'] = data_8nt.PAM.str.split('',expand=True)[5]
# data_8nt['nt6'] = data_8nt.PAM.str.split('',expand=True)[6]
# data_8nt['nt7'] = data_8nt.PAM.str.split('',expand=True)[7]
# data_8nt['nt8'] = data_8nt.PAM.str.split('',expand=True)[8]

# data_8nt.to_csv('/content/drive/My Drive/team_5/project_1/Data/Final_Cleaned_Dataset.csv')



The data to be used in this project has been cleaned and exported as a csv named 'Final_Cleaned_Dataset.csv'

In [ ]:
data_8nt = pd.read_csv('/content/drive/My Drive/BME590 - Deep Learning for Protein Design - Spring 2023/Teams/team_5/project_1/Final_Cleaned_Dataset.csv')

In [ ]:
protseqs = list(data_8nt['AA Sequence'])
seq_len = list(data_8nt['Sequence Length'])
max_seq_len = np.max(seq_len)

#Protein Embedding
Below are the various embedding algorithms we implemented and played around with. **We ultimately decided to only use the VHSE embedding** as it contains information about features (such as hydrophobicity, sterics and electronic properties) that we felt are most important to Cas9 binding to DNA sequences

## VHSE Embedding 
This function takes in an amino acid sequence, splits the string into its component characters, and assembles a VHSE encoded matrix. Each row of the matrix represents one amino acid in the protein sequence, and there are 8 columns for each of the 8 VHSE values. The function also takes as an input the maximum length of a protein sequence so that adequate zero-padding can be applied to make sure all inputs are of the same size.

In [ ]:
#Using pandas to read in the csv file containing the VHSE encoding values for each amino acid
VHSE_val = pd.read_csv('/content/drive/My Drive/BME590 - Deep Learning for Protein Design - Spring 2023/Teams/team_5/project_1/VHSE8.csv')

def VHSE_encode(seq, max_len):
    s = [*seq]
    m = []
    for i in range(0,len(s)):
        for l in range(0,np.shape(VHSE_val)[0]):
            if s[i] == VHSE_val['Single Code'][l]:
                m.append(VHSE_val.iloc[l][2:10]) #This adds all 8 VHSE values for a given amino acid
    
    if len(m) != max_len:
       for x in range(0,max_len-len(m)):
         m.append([0,0,0,0,0,0,0,0])

    a = np.array(m).astype('float32')

    return a

The VHSE takes approximately 30 minutes to run due to the large volume of data. Thus, instead of having to re-embed the protein sequences every time we start a new runtime, the VHSE encoded protein sequences were saved as a csv file that could then be loaded and reshaped into the 3D array

In [ ]:
#VHSE_encoded = [VHSE_encode(seq,max_seq_len) for seq in protseqs]
#VHSE_encoded = np.array(VHSE_encoded)

The two functions defined below allow for the export and import of the VHSE-embedded csv file.

In [ ]:
def export_matrix(a,file):
    a_reshaped = a.reshape(a.shape[0],-1)
    np.savetxt(file,a_reshaped)

def import_matrix(file,twoshape):
  loaded_a = np.loadtxt(file)
  load_original_a = loaded_a.reshape(loaded_a.shape[0],loaded_a.shape[1] // twoshape, twoshape)
  return load_original_a

This code block has been commented because the csv export has already happened

In [ ]:
 #export_matrix(VHSE_encoded,'/content/drive/My Drive/team_5/project_1/Data/FinalData_VHSE_Encoded.csv')

The final embedding result is a 3D array, where each sample is encoded as a 1679 x 8 matrix

In [ ]:
VHSE_encoded = import_matrix('/content/drive/My Drive/BME590 - Deep Learning for Protein Design - Spring 2023/Teams/team_5/project_1/FinalData_VHSE_Encoded.csv',8)

## Bag of Words Embedding
This encoding involves assigning each amino acid a number, so that a protein sequence can be translated into set of values where each number ranges from 1-20 (since there are 20 amino acids). Thus, the final result of this embedding is a 2D array of size n x l, where n is the number of samples, and l the maximum amino acid sequence length.

In [ ]:
amino_acids = [' ', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L',
               'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
len(amino_acids)

21

In [ ]:
def BoW_encoding(seq, max_len):
  encoding = []
  for aa in seq:
    if aa in amino_acids:
      encoding.append(amino_acids.index(aa))
  while(len(encoding) != max_len):
    encoding.append(0)
  return np.array(encoding)

In [ ]:
BoW_seq = [BoW_encoding(seq,max_seq_len) for seq in protseqs]

## BLOSUM62 Embedding
This encoding is implemented in a manner similar to the VHSE embedding. However, the values for each amino acid in BLOSUM62 represent the evolutionary distance of the amino acid to every other. The final result is a 3D array of size 20 x l, where 20 represents the 20 blosum values for each amino acid and l is the maximum protein sequence length in the dataset.


In [ ]:
blosum62 = pd.read_csv('/content/drive/My Drive/team_5/project_1/Unclean Data/BLOSUM62.csv', index_col=0)
blosum62 = blosum62.drop(['B','X','Z','*'],axis=1)

In [ ]:
#encode a peptide into blosum features
def blosum_encode(seq):
    s = [*seq]
    x = pd.DataFrame()
    for i in range(0,len(s)):
      for l in range(0,np.shape(blosum62)[1]):
        if s[i] == blosum62.columns[l]:
          x[i] = blosum62.iloc[l]

    if len(x.columns) != max_seq_len:
       for f in range(0,max_seq_len-len(x.columns)):
         x.loc[len(x)] = 0 # 0 pad for columns

   
    d = np.array(x).astype('float32')
    print(d)
    e = x.values.flatten()  
    return d

In [ ]:
blosum_protseq = [blosum_encode(seq) for seq in protseqs]

# Model Training
Below are the models we tried to use. We played around with a Convolutional Neural Network, Support Vector Machines, Logisitic Regressions, and a Boosted Decision Tree. Ultimately all of these models were able to predict a particular nucleotide in the PAM sequence with decent accuracy.

## CNN
8 CNNs using the same model architecture were run: each time the VHSE-embedded protein inputs reamined the same, but the 2D PAM labels were swapped out for each run of the model.

###Data Preparation for CNN

#### PAM Encoding for CNN
This section contains the code by which PAM sequences were encoded in n x 4 matrices for use as a multi-label output in our 8 CNN models (1 model for each nucleotide position).

The below dictionary assigns each of the base 4 nucleotides to a number; this is the column index number that will later be used when constructing the PAM encoded arrays.

In [ ]:
nt_dict = {'A':0,'C':1,'T':2,'G':3}

In [ ]:
#Create list for each nucleotide in PAM sequence
nt1 = data_8nt['nt1'].tolist()
nt2 = data_8nt['nt2'].tolist()
nt3 = data_8nt['nt3'].tolist()
nt4 = data_8nt['nt4'].tolist()
nt5 = data_8nt['nt5'].tolist()
nt6 = data_8nt['nt6'].tolist()
nt7 = data_8nt['nt7'].tolist()
nt8 = data_8nt['nt8'].tolist()

The dictionary below assigns all of the IUPAC ambiguity codes to their meanings in terms of the base 4 nucleotides. Thus, coupling this with the expand_PAM function defined below outputs a list with all of the possible DNA sequences only in terms of A,C,T,G even if the input sequence contains an ambiguity code.

In [ ]:
IUPAC_r = {
    'N': '(A|C|T|G)',
    'R': '(G|A)',
    'Y': '(T|C)',
    'M': '(A|C)',
    'K': '(G|T)',
    'S': '(G|C)',
    'W': '(A|T)',
    'H': '(A|C|T)',
    'B': '(G|C|T)',
    'V': '(A|C|G)',
    'D': '(A|G|T)'
}

ambiguity_codes = list(IUPAC_r.keys())

def expand_PAM(seq):
  for s in seq:
    if s in IUPAC_r:
      seq = seq.replace(s, IUPAC_r[s])
  seq = list(exrex.generate(seq))
  return seq

The below function constructs the 2D PAM encoding array for each nucleotide position in the PAM sequence

In [ ]:
def nt_encode(ntlist):
  a = np.zeros((len(ntlist),4))
  for i in range(0,len(ntlist)):
    if type(ntlist[i]) == str:
      if ntlist[i] in ambiguity_codes:
        e = expand_PAM(ntlist[i])
        for x in e:
         a[i][nt_dict[x]] = 1
      else:
        a[i][nt_dict[ntlist[i]]] = 1

  return a

In [ ]:
#Generate 2D PAM encoding arrays for each nucleotide position in the PAM sequence
nt1_encoded = nt_encode(nt1)
nt2_encoded = nt_encode(nt2)
nt3_encoded = nt_encode(nt3)
nt4_encoded = nt_encode(nt4)
nt5_encoded = nt_encode(nt5)
nt6_encoded = nt_encode(nt6)
nt7_encoded = nt_encode(nt7)
nt8_encoded = nt_encode(nt8)

####Reshaping the VHSE Protein Embedding
The VHSE encoded array is reshaped so that it can be a tensor, which is the required input for the CNN models.

In [ ]:
VHSE_encoded_tensor = np.expand_dims(VHSE_encoded,-1)

####Splitting the Data for CNN
The data for each nucleotide position was split as 72/8/20 for train/val/test. The VHSE-encoded protein sequences remain constant across all models, only the PAM nucleotide position labels are changing.

In [ ]:
train_seq, test_seq, train_nt1, test_nt1 = train_test_split(VHSE_encoded_tensor,nt1_encoded, test_size=0.2, random_state = 2)
train_seq, val_seq, train_nt1, val_nt1 = train_test_split(train_seq, train_nt1, test_size=0.1, random_state=42)

train_seq, test_seq, train_nt2, test_nt2 = train_test_split(VHSE_encoded_tensor,nt2_encoded, test_size=0.2, random_state = 2)
train_seq, val_seq, train_nt2, val_nt2 = train_test_split(train_seq, train_nt2, test_size=0.1, random_state=42)

train_seq, test_seq, train_nt3, test_nt3 = train_test_split(VHSE_encoded_tensor,nt3_encoded, test_size=0.2, random_state = 2)
train_seq, val_seq, train_nt3, val_nt3 = train_test_split(train_seq, train_nt3, test_size=0.1, random_state=42)

train_seq, test_seq, train_nt4, test_nt4 = train_test_split(VHSE_encoded_tensor,nt4_encoded, test_size=0.2, random_state = 2)
train_seq, val_seq, train_nt4, val_nt4 = train_test_split(train_seq, train_nt4, test_size=0.1, random_state=42)

train_seq, test_seq, train_nt5, test_nt5 = train_test_split(VHSE_encoded_tensor,nt5_encoded, test_size=0.2, random_state = 2)
train_seq, val_seq, train_nt5, val_nt5 = train_test_split(train_seq, train_nt5, test_size=0.1, random_state=42)

train_seq, test_seq, train_nt6, test_nt6 = train_test_split(VHSE_encoded_tensor,nt6_encoded, test_size=0.2, random_state = 2)
train_seq, val_seq, train_nt6, val_nt6 = train_test_split(train_seq, train_nt6, test_size=0.1, random_state=42)

train_seq, test_seq, train_nt7, test_nt7 = train_test_split(VHSE_encoded_tensor,nt7_encoded, test_size=0.2, random_state = 2)
train_seq, val_seq, train_nt7, val_nt7 = train_test_split(train_seq, train_nt7, test_size=0.1, random_state=42)

train_seq, test_seq, train_nt8, test_nt8 = train_test_split(VHSE_encoded_tensor,nt8_encoded, test_size=0.2, random_state = 2)
train_seq, val_seq, train_nt8, val_nt8 = train_test_split(train_seq, train_nt8, test_size=0.1, random_state=42)

print(np.shape(train_seq))
print(np.shape(train_nt1))

NameError: ignored

###CNN Model Architecture
The CNN model architecutre consists of a convolution layer with 32 kernels of size (3,3), followed by a 2x2 maxpool layer, followed by another convolution layer with 64 kernels of size (3,3), followed by another 2x2 maxpool layer, followed by a flatten layer, dropout of 0.6, and a final dense layer.

In [ ]:
input_shape = (1697,8,1)
num_classes = 4

# CNN for 1st Nucleotide Position

model = keras.Sequential(
    
    [
        keras.Input(shape = input_shape),
        layers.Conv2D(32, kernel_size = (3,3), padding='same',activation="relu"),
        layers.MaxPooling2D(pool_size = (2, 2),padding='same'),
        layers.Conv2D(64, kernel_size = (3,3), padding='same',activation="relu"),
        layers.MaxPooling2D(pool_size = (2, 2),padding='same'),
        layers.Flatten(),
        layers.Dropout(0.6),
        layers.Dense(num_classes, activation="softmax")

    ]

)

model.compile(optimizer='adam', loss='BinaryCrossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 1697, 8, 32)       320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 849, 4, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 849, 4, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 425, 2, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 54400)             0         
                                                                 
 dropout (Dropout)           (None, 54400)             0

###Running the CNN Models
A function was defined so that all 8 CNN models could be run with relative ease. The outputs display the loss and accuracy for the training and validation sets during training, and the test loss and accuracy during the model test.

In [ ]:
def CNN(model,train_input,train_labels,val_input,val_labels,test_input,test_labels):
  history = model.fit(x= train_input,
                      y= train_labels,
                      batch_size=60, 
                      callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)],
                      validation_data = (val_input,val_labels),
                      verbose = 1,
                      epochs = 50)
  
  score = model.evaluate(x = test_input, 
                             y = test_labels,
                             verbose=1)
  
  print('Test loss:', score[0]) 
  print('Test accuracy:', score[1])

In [ ]:
CNN(model,train_seq,train_nt1,val_seq,val_nt1,test_seq,test_nt1)

In [ ]:
CNN(model,train_seq,train_nt2,val_seq,val_nt2,test_seq,test_nt2)

In [ ]:
CNN(model,train_seq,train_nt3,val_seq,val_nt3,test_seq,test_nt3)

In [ ]:
CNN(model,train_seq,train_nt4,val_seq,val_nt4,test_seq,test_nt4)

In [ ]:
CNN(model,train_seq,train_nt5,val_seq,val_nt5,test_seq,test_nt5)

Epoch 1/50
76/76 [==============================] - 115s 2s/step - loss: 0.6594 - accuracy: 0.6364 - val_loss: 0.5651 - val_accuracy: 0.7405
Epoch 2/50
76/76 [==============================] - 112s 1s/step - loss: 0.5537 - accuracy: 0.6948 - val_loss: 0.5180 - val_accuracy: 0.7764
Epoch 3/50
76/76 [==============================] - 115s 2s/step - loss: 0.5215 - accuracy: 0.6930 - val_loss: 0.4928 - val_accuracy: 0.7585
Epoch 4/50
76/76 [==============================] - 114s 2s/step - loss: 0.5036 - accuracy: 0.7103 - val_loss: 0.4838 - val_accuracy: 0.7046
Epoch 5/50
76/76 [==============================] - 112s 1s/step - loss: 0.4921 - accuracy: 0.6861 - val_loss: 0.4703 - val_accuracy: 0.7904
Epoch 6/50
76/76 [==============================] - 114s 1s/step - loss: 0.4797 - accuracy: 0.6985 - val_loss: 0.4628 - val_accuracy: 0.7206
Epoch 7/50
29/76 [==========>...................] - ETA: 1:08 - loss: 0.4714 - accuracy: 0.6868

In [ ]:
CNN(model,train_seq,train_nt6,val_seq,val_nt6,test_seq,test_nt6)

In [ ]:
CNN(model,train_seq,train_nt7,val_seq,val_nt7,test_seq,test_nt7)

Epoch 1/50
76/76 [==============================] - 109s 1s/step - loss: 0.6692 - accuracy: 0.3869 - val_loss: 0.6065 - val_accuracy: 0.5210
Epoch 2/50
76/76 [==============================] - 114s 1s/step - loss: 0.6004 - accuracy: 0.4698 - val_loss: 0.5749 - val_accuracy: 0.5868
Epoch 3/50
76/76 [==============================] - 111s 1s/step - loss: 0.5830 - accuracy: 0.5149 - val_loss: 0.5639 - val_accuracy: 0.5768
Epoch 4/50
76/76 [==============================] - 116s 2s/step - loss: 0.5681 - accuracy: 0.5184 - val_loss: 0.5490 - val_accuracy: 0.6008
Epoch 5/50
76/76 [==============================] - 111s 1s/step - loss: 0.5585 - accuracy: 0.5271 - val_loss: 0.5447 - val_accuracy: 0.5808
Epoch 6/50
76/76 [==============================] - 112s 1s/step - loss: 0.5485 - accuracy: 0.5264 - val_loss: 0.5350 - val_accuracy: 0.6148
Epoch 7/50
76/76 [==============================] - 115s 2s/step - loss: 0.5417 - accuracy: 0.5253 - val_loss: 0.5340 - val_accuracy: 0.5828
Epoch 8/50
76

In [ ]:
CNN(model,train_seq,train_nt8,val_seq,val_nt8,test_seq,test_nt8)

Epoch 1/50
76/76 [==============================] - 109s 1s/step - loss: 0.6820 - accuracy: 0.3321 - val_loss: 0.6311 - val_accuracy: 0.3693
Epoch 2/50
76/76 [==============================] - 107s 1s/step - loss: 0.6155 - accuracy: 0.4028 - val_loss: 0.5541 - val_accuracy: 0.5269
Epoch 3/50
76/76 [==============================] - 107s 1s/step - loss: 0.5883 - accuracy: 0.4627 - val_loss: 0.5448 - val_accuracy: 0.5329
Epoch 4/50
76/76 [==============================] - 110s 1s/step - loss: 0.5652 - accuracy: 0.4984 - val_loss: 0.5406 - val_accuracy: 0.5250
Epoch 5/50
76/76 [==============================] - 107s 1s/step - loss: 0.5567 - accuracy: 0.5060 - val_loss: 0.5320 - val_accuracy: 0.5329
Epoch 6/50
76/76 [==============================] - 109s 1s/step - loss: 0.5582 - accuracy: 0.5049 - val_loss: 0.5263 - val_accuracy: 0.5489
Epoch 7/50
76/76 [==============================] - 108s 1s/step - loss: 0.5526 - accuracy: 0.5197 - val_loss: 0.5313 - val_accuracy: 0.5349
Epoch 8/50
76

##Reshaping The Data For SVM, Logistic Regression, and AdaBoost
These models cannot handle inputs with dimension greater than 2; thus the VHSE-embedded protein data was flattened into a 2D array

In [ ]:
flattened_VHSE = []
for row in VHSE_encoded:
  flattened_VHSE.append(row.flatten('F'))
df = pd.DataFrame(flattened_VHSE)

,0,1,2,3,4,5,6,7,8,9,...,13566,13567,13568,13569,13570,13571,13572,13573,13574,13575
0,1.01,-1.17,-0.99,1.01,-1.17,-0.99,0.22,0.61,-0.34,1.27,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.01,-0.67,-1.15,1.36,0.76,1.36,-0.20,1.36,-1.15,1.27,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.01,-0.20,0.61,-0.34,0.76,-0.20,1.36,-1.15,1.27,-0.20,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.01,-1.17,-1.17,-1.15,0.61,0.76,1.27,-0.20,1.36,-1.15,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.01,-0.34,-1.17,1.27,-1.17,-0.99,-1.18,0.61,1.27,0.76,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6257,1.01,-1.47,-0.20,-1.17,-1.47,0.61,-1.47,1.27,-0.20,1.27,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6258,1.01,-0.99,-1.17,-0.96,1.50,1.01,-0.20,-0.43,-0.34,-1.18,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6259,1.01,-1.15,-1.17,-1.17,0.61,-0.67,1.27,-0.20,1.36,-1.15,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6260,1.01,-1.17,-1.17,1.52,-1.18,-0.99,0.61,0.61,1.36,-0.20,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Additionally, these models cannot handle 2D outputs like the CNN can, so the PAM encoding resulted in defining each IUPAC nucleotide code as a separate class, for a total of 15 classes.

In [ ]:
def encode_nuc(labels):
  label_dict = {
    'A': 0,
    'T': 1,
    'C': 2,
    'G': 3,
    'N': 4,
    'R': 5,
    'Y': 6,
    'M': 7,
    'K': 8,
    'S': 9,
    'W': 10,
    'H': 11,
    'B': 12,
    'V': 13,
    'D': 14,
  }
  ret = []
  next = 15
  for nuc in labels:
      if nuc not in label_dict.keys():
        label_dict[nuc] = next
        next += 1
      ret.append(label_dict[nuc])
  print(len(ret))
  return ret

In [ ]:
nt1_labels = encode_nuc(nt1)
nt2_labels = encode_nuc(nt2)
nt3_labels = encode_nuc(nt3)
nt4_labels = encode_nuc(nt4)
nt5_labels = encode_nuc(nt5)
nt6_labels = encode_nuc(nt6)
nt7_labels = encode_nuc(nt7)
nt8_labels = encode_nuc(nt8)

6262
6262
6262
6262
6262
6262
6262
6262


Data for each nucleotide was split with the ratio 67/33 test/train

In [ ]:
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(df, nt1_labels, test_size=0.33, random_state = 2)
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(df, nt2_labels, test_size=0.33, random_state = 2)
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(df, nt3_labels, test_size=0.33, random_state = 2)
X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(df, nt4_labels, test_size=0.33, random_state = 2)
X_train_5, X_test_5, y_train_5, y_test_5 = train_test_split(df, nt5_labels, test_size=0.33, random_state = 2)
X_train_6, X_test_6, y_train_6, y_test_6 = train_test_split(df, nt6_labels, test_size=0.33, random_state = 2)
X_train_7, X_test_7, y_train_7, y_test_7 = train_test_split(df, nt7_labels, test_size=0.33, random_state = 2)
X_train_8, X_test_8, y_train_8, y_test_8 = train_test_split(df, nt8_labels, test_size=0.33, random_state = 2)

## Logistic Regression
Scikit-learn Logistic Regression was trained and tested for each nucleotide position with 100 estimators

In [ ]:
logReg = LogisticRegression(random_state=0, max_iter=100).fit(X_train_1, y_train_1)
test_preds_1 = logReg.predict(X_test_1)
train_preds_1 = logReg.predict(X_train_1)
print(accuracy_score(y_train_1, train_preds_1), accuracy_score(y_test_1, test_preds_1))

In [ ]:
logReg = LogisticRegression(random_state=0, max_iter=100).fit(X_train_2, y_train_2)
test_preds_2 = logReg.predict(X_test_2)
train_preds_2 = logReg.predict(X_train_2)
print(accuracy_score(y_train_2, train_preds_2), accuracy_score(y_test_2, test_preds_2))

In [ ]:
logReg = LogisticRegression(random_state=0, max_iter=100).fit(X_train_3, y_train_3)
test_preds_3 = logReg.predict(X_test_3)
train_preds_3 = logReg.predict(X_train_3)
print(accuracy_score(y_train_3, train_preds_3), accuracy_score(y_test_3, test_preds_3))

In [ ]:
logReg = LogisticRegression(random_state=0, max_iter=100).fit(X_train_4, y_train_4)
test_preds_4 = logReg.predict(X_test_4)
train_preds_4 = logReg.predict(X_train_4)
print(accuracy_score(y_train_4, train_preds_4), accuracy_score(y_test_4, test_preds_4))

In [ ]:
logReg = LogisticRegression(random_state=0, max_iter=100).fit(X_train_5, y_train_5)
test_preds_5 = logReg.predict(X_test_5)
train_preds_5 = logReg.predict(X_train_5)
print(accuracy_score(y_train_5, train_preds_5), accuracy_score(y_test_5, test_preds_5))

In [ ]:
logReg = LogisticRegression(random_state=0, max_iter=100).fit(X_train_6, y_train_6)
test_preds_6 = logReg.predict(X_test_6)
train_preds_6 = logReg.predict(X_train_6)
print(accuracy_score(y_train_6, train_preds_6), accuracy_score(y_test_6, test_preds_6))

In [ ]:
logReg = LogisticRegression(random_state=0, max_iter=100).fit(X_train_7, y_train_7)
test_preds_7 = logReg.predict(X_test_7)
train_preds_7 = logReg.predict(X_train_7)
print(accuracy_score(y_train_7, train_preds_7), accuracy_score(y_test_7, test_preds_7))

In [ ]:
logReg = LogisticRegression(random_state=0, max_iter=100).fit(X_train_8, y_train_8)
test_preds_8 = logReg.predict(X_test_8)
train_preds_8 = logReg.predict(X_train_8)
print(accuracy_score(y_train_8, train_preds_8), accuracy_score(y_test_8, test_preds_8))

## SVM
Support Vector Machines for each of the 8 nucleotide positions were run with and without grid search for hyperparameter optimization

Doing Cross Validation Grid Search with k=3

In [ ]:
param_grid = [
  {'C': [0.1, 1, 100], 'kernel': ['rbf'], 'gamma': [0.001, 0.0001, 'scale'], 'verbose' : [True]}
 ]

svm = SVC()
clf = GridSearchCV(svm, param_grid, cv=3)

In [ ]:
clf.fit(X_train_1, y_train_1)
test_preds_1 = clf.predict(X_test_1)
train_preds_1 = clf.predict(X_train_1)
print("Model 1 Train, Test Accuracy \n")
print(accuracy_score(y_train_1, train_preds_1), accuracy_score(y_test_1, test_preds_1))

In [ ]:
clf.fit(X_train_2, y_train_2)
test_preds_2 = clf.predict(X_test_2)
train_preds_2 = clf.predict(X_train_2)
print("Model 2 Train, Test Accuracy \n")
print(accuracy_score(y_train_2, train_preds_2), accuracy_score(y_test_2, test_preds_2))

In [ ]:
clf.fit(X_train_3, y_train_3)
test_preds_3 = clf.predict(X_test_3)
train_preds_3 = clf.predict(X_train_3)
print("Model  Train, Test Accuracy \n")
print(accuracy_score(y_train_3, train_preds_3), accuracy_score(y_test_3, test_preds_3))

In [ ]:
clf.fit(X_train_4, y_train_4)
test_preds_4 = clf.predict(X_test_4)
train_preds_4 = clf.predict(X_train_4)
print("Model " + 4 + " Train, Test Accuracy \n")
print(accuracy_score(y_train_4, train_preds_4), accuracy_score(y_test_4, test_preds_4))

In [ ]:
clf.fit(X_train_5, y_train_5)
test_preds_5 = clf.predict(X_test_5)
train_preds_5 = clf.predict(X_train_5)
print("Model " + 5 + " Train, Test Accuracy \n")
print(accuracy_score(y_train_5, train_preds_5), accuracy_score(y_test_5, test_preds_5))

In [ ]:
clf.fit(X_train_6, y_train_6)
test_preds_6 = clf.predict(X_test_6)
train_preds_6 = clf.predict(X_train_6)
print("Model " + 6 + " Train, Test Accuracy \n")
print(accuracy_score(y_train_6, train_preds_6), accuracy_score(y_test_6, test_preds_6))

In [ ]:
clf.fit(X_train_7, y_train_7)
test_preds_7 = clf.predict(X_test_7)
train_preds_7 = clf.predict(X_train_7)
print("Model " + 7 + " Train, Test Accuracy \n")
print(accuracy_score(y_train_7, train_preds_7), accuracy_score(y_test_7, test_preds_7))

In [ ]:
clf.fit(X_train_8, y_train_8)
test_preds_8 = clf.predict(X_test_8)
train_preds_8 = clf.predict(X_train_8)
print("Model " + 8 + " Train, Test Accuracy \n")
print(accuracy_score(y_train_8, train_preds_8), accuracy_score(y_test_8, test_preds_8))

Now with no cross validation (just built in SKLearn)

In [ ]:
svm.fit(X_train_1, y_train_1)
test_preds_1 = svm.predict(X_test_1)
train_preds_1 = svm.predict(X_train_1)
print(accuracy_score(y_train_1, train_preds_1), accuracy_score(y_test_1, test_preds_1))

In [ ]:
svm.fit(X_train_2, y_train_2)
test_preds_2 = svm.predict(X_test_2)
train_preds_2 = svm.predict(X_train_2)
print(accuracy_score(y_train_2, train_preds_2), accuracy_score(y_test_2, test_preds_2))

In [ ]:
svm.fit(X_train_3, y_train_3)
test_preds_3 = svm.predict(X_test_3)
train_preds_3 = svm.predict(X_train_3)
print(accuracy_score(y_train_3, train_preds_3), accuracy_score(y_test_3, test_preds_3))

In [ ]:
svm.fit(X_train_4, y_train_4)
test_preds_4 = svm.predict(X_test_4)
train_preds_4 = svm.predict(X_train_4)
print(accuracy_score(y_train_4, train_preds_4), accuracy_score(y_test_4, test_preds_4))

In [ ]:
svm.fit(X_train_5, y_train_5)
test_preds_5 = svm.predict(X_test_5)
train_preds_5 = svm.predict(X_train_5)
print(accuracy_score(y_train_5, train_preds_5), accuracy_score(y_test_5, test_preds_5))

In [ ]:
svm.fit(X_train_6, y_train_6)
test_preds_6 = svm.predict(X_test_6)
train_preds_6 = svm.predict(X_train_6)
print(accuracy_score(y_train_6, train_preds_6), accuracy_score(y_test_6, test_preds_6))

In [ ]:
svm.fit(X_train_7, y_train_7)
test_preds_7 = svm.predict(X_test_7)
train_preds_7 = svm.predict(X_train_7)
print(accuracy_score(y_train_7, train_preds_7), accuracy_score(y_test_7, test_preds_7))

In [ ]:
svm.fit(X_train_8, y_train_8)
test_preds_8 = svm.predict(X_test_8)
train_preds_8 = svm.predict(X_train_8)
print(accuracy_score(y_train_8, train_preds_8), accuracy_score(y_test_8, test_preds_8))

## AdaBoost
Sci-kit learn AdaBoost Classifier was run for all 8 nucleotide positions with 200 estimators.

In [ ]:
AdaBoost = AdaBoostClassifier(n_estimators=200, random_state=0)
AdaBoost.fit(X_test_1, y_test_1)
test_preds_1 = AdaBoost.predict(X_test_1)
train_preds_1 = AdaBoost.predict(X_train_1)
print(accuracy_score(y_train_1, train_preds_1), accuracy_score(y_test_1, test_preds_1))

In [ ]:
AdaBoost = AdaBoostClassifier(n_estimators=200, random_state=0)
AdaBoost.fit(X_test_2, y_test_2)
test_preds = AdaBoost.predict(X_test_2)
train_preds = AdaBoost.predict(X_train_2)
print(accuracy_score(y_train_2, train_preds), accuracy_score(y_test_2, test_preds))

In [ ]:
AdaBoost = AdaBoostClassifier(n_estimators=200, random_state=0)
AdaBoost.fit(X_test_3, y_test_3)
test_preds = AdaBoost.predict(X_test_3)
train_preds = AdaBoost.predict(X_train_3)
print(accuracy_score(y_train_3, train_preds), accuracy_score(y_test_3, test_preds))

In [ ]:
AdaBoost = AdaBoostClassifier(n_estimators=200, random_state=0)
AdaBoost.fit(X_test_4, y_test_4)
test_preds = AdaBoost.predict(X_test_4)
train_preds = AdaBoost.predict(X_train_4)
print(accuracy_score(y_train_4, train_preds), accuracy_score(y_test_4, test_preds))

In [ ]:
AdaBoost = AdaBoostClassifier(n_estimators=200, random_state=0)
AdaBoost.fit(X_test_5, y_test_5)
test_preds = AdaBoost.predict(X_test_5)
train_preds = AdaBoost.predict(X_train_5)
print(accuracy_score(y_train_5, train_preds), accuracy_score(y_test_5, test_preds))

In [ ]:
AdaBoost = AdaBoostClassifier(n_estimators=200, random_state=0)
AdaBoost.fit(X_test_6, y_test_6)
test_preds = AdaBoost.predict(X_test_6)
train_preds = AdaBoost.predict(X_train_6)
print(accuracy_score(y_train_6, train_preds), accuracy_score(y_test_6, test_preds))

In [ ]:
AdaBoost = AdaBoostClassifier(n_estimators=200, random_state=0)
AdaBoost.fit(X_test_7, y_test_7)
test_preds = AdaBoost.predict(X_test_7)
train_preds = AdaBoost.predict(X_train_7)
print(accuracy_score(y_train_7, train_preds), accuracy_score(y_test_7, test_preds))

In [ ]:
AdaBoost = AdaBoostClassifier(n_estimators=200, random_state=0)
AdaBoost.fit(X_test_8, y_test_8)
test_preds = AdaBoost.predict(X_test_8)
train_preds = AdaBoost.predict(X_train_8)
print(accuracy_score(y_train_8, train_preds), accuracy_score(y_test_8, test_preds))

#Trying ANKH Encoding
This simply represents an attempt to embed the protein sequences using Ankh, this code does not execute without errors.

In [ ]:
!pip install ankh

In [ ]:
  import ankh

  # To load large model:
  model, tokenizer = ankh.load_large_model()
  model.eval()

In [ ]:
!pip install torch
import torch

feature extraction using large model

In [ ]:
  model, tokenizer = ankh.load_large_model()
  model.eval()

  protein_sequences = protseqs

  protein_sequences = [list(seq) for seq in protein_sequences]

In [ ]:
type(model)

In [ ]:
outputs = tokenizer.batch_encode_plus(protein_sequences, 
                                    add_special_tokens=True, 
                                    padding=True, 
                                    is_split_into_words=True, 
                                    return_tensors="pt")

load model and number of parameters

In [ ]:
def get_num_params(model):
    return sum(p.numel() for p in model.parameters())

In [ ]:
from torch import nn
from torch.utils.data import Dataset, DataLoader
import ankh
from transformers import Trainer, TrainingArguments, EvalPrediction
from datasets import load_dataset
import transformers.models.convbert as c_bert
from scipy import stats
from functools import partial
import pandas as pd
from tqdm.auto import tqdm

In [ ]:
import os
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [ ]:
#Select the available device.
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cuda')
print('Available device:', device)

In [ ]:
# imports are always needed
import torch

In [ ]:
# get index of currently selected device
torch.cuda.current_device() # returns 0 in my case

In [ ]:
# get number of GPUs available
torch.cuda.device_count() # returns 1 in my case

In [ ]:
# get the name of the device
torch.cuda.get_device_name(0) # good old Tesla K80

In [ ]:
model, tokenizer = ankh.load_large_model()
model.eval()
model.to(device=device)

In [ ]:
print(f"Number of parameters:", get_num_params(model))

Load the datasets

In [ ]:
data8 = pd.read_csv('/content/drive/My Drive/team_5/project_1/Data/PAM_data8_ranked.csv')
data_8nt = pd.DataFrame()
data_8nt['AA Sequence'] = data8['Sequence']
data_8nt['Sequence Length'] = data8['Length']
data_8nt['PAM'] = data8['consensus PAM']

data_8nt['nt1'] = data_8nt.PAM.str.split('',expand=True)[1]
data_8nt['nt2'] = data_8nt.PAM.str.split('',expand=True)[2]
data_8nt['nt3'] = data_8nt.PAM.str.split('',expand=True)[3]
data_8nt['nt4'] = data_8nt.PAM.str.split('',expand=True)[4]
data_8nt['nt5'] = data_8nt.PAM.str.split('',expand=True)[5]
data_8nt['nt6'] = data_8nt.PAM.str.split('',expand=True)[6]
data_8nt['nt7'] = data_8nt.PAM.str.split('',expand=True)[7]
data_8nt['nt8'] = data_8nt.PAM.str.split('',expand=True)[8]

data_8nt


View dataset from the ankh github

In [ ]:
from datasets import load_dataset
dataset = load_dataset("proteinea/Fluorosence")

In [ ]:
dataset

In [ ]:
dataset['train']['log_fluorescence']

split data in train,test,validation

In [ ]:
aa_data = data_8nt['AA Sequence']
pam_data = data_8nt['PAM']
nt1 = data_8nt['nt1']
nt2 = data_8nt['nt2']
nt3 = data_8nt['nt3']
nt4 = data_8nt['nt4']
nt5 = data_8nt['nt5']
nt6 = data_8nt['nt6']
nt7 = data_8nt['nt7']
nt8 = data_8nt['nt8']

In [ ]:
nt1

In [ ]:
# full pam sequence --> taking insanely long for embedding (23 hrs)
train_seq, test_seq, train_pam, test_pam = train_test_split(aa_data,pam_data, test_size=0.2, random_state = 2)
train_seq, val_seq, train_pam, val_pam = train_test_split(aa_data, pam_data, test_size=0.1, random_state=42)

In [ ]:
# Get the mean of the labels to initialize 
# the final layer's bias with it for faster convergence in regression tasks.
training_labels_mean = np.mean(pam_data)
training_labels_mean

In [ ]:
# each individual nucleotide (in the pam)
train_seq, test_seq, train_nt1, test_nt1 = train_test_split(aa_data,nt1_encoded, test_size=0.2, random_state = 2)
train_seq, val_seq, train_nt1, val_nt1 = train_test_split(train_seq, train_nt1, test_size=0.1, random_state=42)

In [ ]:
train_seq, test_seq, train_nt2, test_nt2 = train_test_split(aa_data,nt2, test_size=0.2, random_state = 2)
train_seq, val_seq, train_nt2, val_nt2 = train_test_split(train_seq, train_nt2, test_size=0.1, random_state=42)

In [ ]:
train_seq, test_seq, train_nt3, test_nt3 = train_test_split(aa_data,nt3, test_size=0.2, random_state = 2)
train_seq, val_seq, train_nt3, val_nt3 = train_test_split(train_seq, train_nt3, test_size=0.1, random_state=42)

In [ ]:
train_seq, test_seq, train_nt4, test_nt4 = train_test_split(aa_data,nt4, test_size=0.2, random_state = 2)
train_seq, val_seq, train_nt4, val_nt4 = train_test_split(train_seq, train_nt4, test_size=0.1, random_state=42)

In [ ]:
train_seq, test_seq, train_nt5, test_nt5 = train_test_split(aa_data,nt5, test_size=0.2, random_state = 2)
train_seq, val_seq, train_nt5, val_nt5 = train_test_split(train_seq, train_nt5, test_size=0.1, random_state=42)

In [ ]:
train_seq, test_seq, train_nt6, test_nt6 = train_test_split(aa_data,nt6, test_size=0.2, random_state = 2)
train_seq, val_seq, train_nt6, val_nt6 = train_test_split(train_seq, train_nt6, test_size=0.1, random_state=42)

In [ ]:
train_seq, test_seq, train_nt7, test_nt7 = train_test_split(aa_data,nt7, test_size=0.2, random_state = 2)
train_seq, val_seq, train_nt7, val_nt7 = train_test_split(train_seq, train_nt7, test_size=0.1, random_state=42)

In [ ]:
train_seq, test_seq, train_nt8, test_nt8 = train_test_split(aa_data,nt8, test_size=0.2, random_state = 2)
train_seq, val_seq, train_nt8, val_nt8 = train_test_split(train_seq, train_nt8, test_size=0.1, random_state=42)

preprocess dataset

In [ ]:
def preprocess_dataset(sequences, labels, max_length=None):
    '''
        Args:
            sequences: list, the list which contains the protein primary sequences.
            max_length, Integer, the maximum sequence length, 
            if there is a sequence that is larger than the specified sequence length will be post-truncated. 
    '''
    if max_length is None:
        max_length = len(max(train_seq, key=lambda x: len(x)))
    splitted_sequences = [list(seq[:max_length]) for seq in sequences]
    return splitted_sequences

In [ ]:
# train_seq
train_seq_1 = preprocess_dataset(train_seq, train_nt1)
train_seq_2 = preprocess_dataset(train_seq, train_nt2)
train_seq_3 = preprocess_dataset(train_seq, train_nt3)
train_seq_4 = preprocess_dataset(train_seq, train_nt4)
train_seq_5 = preprocess_dataset(train_seq, train_nt5)
train_seq_6 = preprocess_dataset(train_seq, train_nt6)
train_seq_7 = preprocess_dataset(train_seq, train_nt7)
train_seq_8 = preprocess_dataset(train_seq, train_nt8)

In [ ]:
# test_seq
test_seq_1 = preprocess_dataset(test_seq, test_nt1)
test_seq_2 = preprocess_dataset(test_seq, test_nt2)
test_seq_3 = preprocess_dataset(test_seq, test_nt3)
test_seq_4 = preprocess_dataset(test_seq, test_nt4)
test_seq_5 = preprocess_dataset(test_seq, test_nt5)
test_seq_6 = preprocess_dataset(test_seq, test_nt6)
test_seq_7 = preprocess_dataset(test_seq, test_nt7)
test_seq_8 = preprocess_dataset(test_seq, test_nt8)

In [ ]:
# val_seq
val_seq_1 = preprocess_dataset(val_seq, val_nt1)
val_seq_2 = preprocess_dataset(val_seq, val_nt2)
val_seq_3 = preprocess_dataset(val_seq, val_nt3)
val_seq_4 = preprocess_dataset(val_seq, val_nt4)
val_seq_5 = preprocess_dataset(val_seq, val_nt5)
val_seq_6 = preprocess_dataset(val_seq, val_nt6)
val_seq_7 = preprocess_dataset(val_seq, val_nt7)
val_seq_8 = preprocess_dataset(val_seq, val_nt8)

In [ ]:
# full pam sequence --> taking insanely long for embedding (23 hrs) -- so is train_seq by nucleotide
train_seq = preprocess_dataset(train_seq, train_pam)
test_seq = preprocess_dataset(test_seq, test_pam)
val_seq = preprocess_dataset(val_seq, val_pam)

extract sequences embeddings

In [ ]:
def embed_dataset(model, sequences, shift_left = 0, shift_right = -1):
    inputs_embedding = []
    with torch.no_grad():
        for sample in tqdm(sequences):
            ids = tokenizer.batch_encode_plus([sample], add_special_tokens=True, 
                                              padding=True, is_split_into_words=True, 
                                              return_tensors="pt")
            embedding = model(input_ids=ids['input_ids'].to(device))[0]
            embedding = embedding[0].detach().cpu().numpy()[shift_left:shift_right]
            inputs_embedding.append(embedding)
    return inputs_embedding

In [ ]:
training_embeddings = embed_dataset(model, train_seq)

In [ ]:
validation_embeddings = embed_dataset(model, val_seq)
test_embeddings = embed_dataset(model, test_seq)

sequence embeddings with pam (labels)

In [ ]:
class pamdataset(Dataset):
    def __init__(self, sequences, labels):
        self.sequences = sequences
        self.labels = labels

    def __getitem__(self, idx):
        sample = self.sequences[idx]
        label = self.labels[idx]
        return {
            'embed': torch.tensor(sample),
            'labels': torch.tensor(label, dtype=torch.float32).unsqueeze(-1)}

    def __len__(self):
        return len(self.labels)

In [ ]:
training_dataset = pamdataset(training_embeddings, train_pam)
validation_dataset = pamdataset(validation_embeddings, val_pam)
test_ds = pamdataset(test_embeddings, test_pam)

In [ ]:
training_embeddings

downstream multiclass classification model

In [ ]:
def model_init(embed_dim, training_labels_mean=None):
    hidden_dim = int(embed_dim / 2)
    num_hidden_layers = 1
    nlayers = 1
    nhead = 4
    dropout = 0.2
    conv_kernel_size = 7
    pooling = 'max' # available pooling methods ['avg', 'max']
    downstream_model = ankh.ConvBertForMultiClassClassification(input_dim=embed_dim, 
                                                  nhead=nhead, 
                                                  hidden_dim=hidden_dim, 
                                                  num_hidden_layers=num_hidden_layers, 
                                                  num_layers=nlayers, 
                                                  kernel_size=conv_kernel_size,
                                                  dropout=dropout, 
                                                  pooling=pooling, 
                                                  training_labels_mean=training_labels_mean)
    return downstream_model.cuda()

In [ ]:
def compute_metrics(p: EvalPrediction):
    return {
        "spearmanr": stats.spearmanr(p.label_ids, p.predictions).correlation,
    }

In [ ]:
type(protseqs)

In [ ]:
model_type = 'ankh_large'
experiment = f'flu_{model_type}'

training_args = TrainingArguments(
    output_dir=f'./results_{experiment}',
    num_train_epochs=5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    warmup_steps=1000,
    learning_rate=1e-03,
    weight_decay=0.0,
    logging_dir=f'./logs_{experiment}',
    logging_steps=200,
    do_train=True,
    do_eval=True,
    evaluation_strategy="epoch",
    gradient_accumulation_steps=16,
    fp16=False,
    fp16_opt_level="02",
    run_name=experiment,
    seed=42,
    load_best_model_at_end=True,
    metric_for_best_model="eval_spearmanr",
    greater_is_better=True,
    save_strategy="epoch"
)

In [ ]:
model_embed_dim = 1536

trainer = Trainer(
    model_init=partial(model_init, embed_dim=model_embed_dim),
    args=training_args,
    train_dataset=training_dataset,
    eval_dataset=validation_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
model_

In [ ]:
model_embed_dim

In [ ]:
trainer.train()


### POSITIONAL ENCODING (COS) + Embedding -- RUN ANKH MODEL

The formula for calculating the positional encoding (implemented in Python below) is as follows:

$$\Large{PE_{(pos, 2i)} = \sin(pos / 10000^{2i / d_{model}})} $$
$$\Large{PE_{(pos, 2i+1)} = \cos(pos / 10000^{2i / d_{model}})} $$

In [ ]:
def positional_encoding(length, depth):
  depth = depth/2

  positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
  depths = np.arange(depth)[np.newaxis, :]/depth   # (1, depth)
  
  angle_rates = 1 / (10000**depths)         # (1, depth)
  angle_rads = positions * angle_rates      # (pos, depth)

  pos_encoding = np.concatenate(
      [np.sin(angle_rads), np.cos(angle_rads)],
      axis=-1) 

  return tf.cast(pos_encoding, dtype=tf.float32)

The position encoding function is a stack of sines and cosines that vibrate at different frequencies depending on their location along the depth of the embedding vector. They vibrate across the position axis.

In [ ]:
#@title
pos_encoding = positional_encoding(length=2048, depth=512)

# Check the shape.
print(pos_encoding.shape)

# Plot the dimensions.
plt.pcolormesh(pos_encoding.numpy().T, cmap='RdBu')
plt.ylabel('Depth')
plt.xlabel('Position')
plt.colorbar()
plt.show()

By definition these vectors align well with nearby vectors along the position axis. Below the position encoding vectors are normalized and the vector from position `1000` is compared, by dot-product, to all the others:

In [ ]:
#@title
pos_encoding/=tf.norm(pos_encoding, axis=1, keepdims=True)
p = pos_encoding[1000]
dots = tf.einsum('pd,d -> p', pos_encoding, p)
plt.subplot(2,1,1)
plt.plot(dots)
plt.ylim([0,1])
plt.plot([950, 950, float('nan'), 1050, 1050],
         [0,1,float('nan'),0,1], color='k', label='Zoom')
plt.legend()
plt.subplot(2,1,2)
plt.plot(dots)
plt.xlim([950, 1050])
plt.ylim([0,1])
